In [1]:
import re
import time
def get_page(str1):
    print("正在下载:", str1)
    time.sleep(2)
    print("下载成功:", str1)
name_list = ['aa', 'bb', 'cc']
start_time = time.time()
for item in name_list:  # 单线程
    get_page(item)
end_time = time.time()
print('%f second' % (end_time - start_time))

正在下载: aa
下载成功: aa
正在下载: bb
下载成功: bb
正在下载: cc
下载成功: cc
6.025902 second


In [2]:
from multiprocessing.dummy import Pool
start_time = time.time()
pool = Pool(3)
pool.map(get_page, name_list)
end_time = time.time()
print('%f second' % (end_time - start_time))

正在下载: aa
正在下载: bb
正在下载: cc
下载成功:下载成功: cc
下载成功: bb
 aa
2.017121 second


In [3]:
import requests
import random
from lxml import etree
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54'
}
url = 'https://www.pearvideo.com/category_5'
urls = []
page_text = requests.get(url=url, headers=header).text
tree = etree.HTML(page_text)
li_list = tree.xpath('//ul[@id="listvideoListUl"]/li')
for li in li_list:
    detail_url = 'https://www.pearvideo.com/' + li.xpath('./div/a/@href')[0]
    name = li.xpath('./div/a/div[2]/text()')[0] + '.mp4'
    print(detail_url, name)
    contId = re.findall('_(.*)', detail_url)[0]
    # print(contId)
    mrd = random.random()
    params = {
            "contId": contId,
            "mrd": mrd
    }
    video_url = "https://www.pearvideo.com/videoStatus.jsp"
    header["Referer"] = detail_url
    video_response = requests.get(url=video_url, headers=header, params=params).text
    # print(video_response)
    fake_video_url = re.findall('"srcUrl":"(.*?)"', video_response)[0]
    # print(fake_video_url)
    url1 = re.findall('https://.*?/.*?/.*?/.*?/(.*?)-', fake_video_url)[0]
    real_video_url = re.sub(url1, 'cont-' + contId, fake_video_url)
    print(real_video_url)
    dic = {
        'name': name,
        'url': real_video_url
    }
    urls.append(dic)

def get_video(dic2):
    url2 = dic2['url']
    header2 = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54'
    }
    data = requests.get(url=url2, headers=header2).content
    with open(dic2['name'], 'wb') as fp:
        fp.write(data)
        print(dic2['name'])

pool = Pool(4)
pool.map(get_video, urls)
pool.close()
pool.join()


https://www.pearvideo.com/video_1383732 他用竹编做二维码，200一个受热捧.mp4
https://video.pearvideo.com/mp4/adshort/20180708/cont-1383732-11983629-185548_adpkg-ad_hd.mp4
https://www.pearvideo.com/video_1383862 大叔卖瓜给娃治病，巡防送水直抹泪.mp4
https://video.pearvideo.com/mp4/adshort/20180708/cont-1383862-12412559_adpkg-ad_hd.mp4
https://www.pearvideo.com/video_1383878 环卫工半天喝5斤水:挣钱给老伴养老.mp4
https://video.pearvideo.com/mp4/adshort/20180708/cont-1383878-12412722_adpkg-ad_hd.mp4
https://www.pearvideo.com/video_1383592 7旬老太为18岁养女寻亲：怕她孤单.mp4
https://video.pearvideo.com/mp4/adshort/20180707/cont-1383592-12409703_adpkg-ad_hd.mp4
大叔卖瓜给娃治病，巡防送水直抹泪.mp4
他用竹编做二维码，200一个受热捧.mp4
7旬老太为18岁养女寻亲：怕她孤单.mp4
环卫工半天喝5斤水:挣钱给老伴养老.mp4
